## Conversion of Images to 8-bit for Applying the RORPO Filter

This code block was developed to convert medical images in NIfTI format (`.nii` or `.nii.gz`) to 8-bit format, a necessary requirement for applying the **RORPO (Ranking Orientation Responses of Path Operators)** filter. This filter only works with 8-bit images, so properly scaling the intensity values of the original images is essential.

### Code Description:

1. **Directory setup**:
   - The root directory is defined where the patient folders (e.g., `AAA417`, etc.) are located.
   - An output folder called `Images_8bits` is created to store the results.

2. **Function `scale_to_uint8`**:
   - Scales the image intensities to the range [0, 255] and converts them to `uint8` type.

3. **Iterating through patient folders**:
   - Only valid directories are processed (the output folder is skipped).
   - For each `.nii` or `.nii.gz` image:
     - The image is loaded using `nibabel`.
     - The scaling function is applied.
     - The new image is saved in the output folder, preserving the affine transformation matrix and header information.

4. **Status messages**:
   - Progress information is printed for each processed file, and any errors encountered are reported.

At the end, all 8-bit converted images will be available in the `Images_8bits` folder, ready to be used in processing with RORPO.

In [ ]:
import os
import nibabel as nib
import numpy as np

# --- Paths ---
root_dir = 'path/to/your/directory'  # Change this to the path where the patient folders like AAA417 are located
output_root = os.path.join(root_dir, 'Images_8bits')
os.makedirs(output_root, exist_ok=True)

# --- Function to scale intensities to 8-bit ---
def scale_to_uint8(data):
    data_min = np.min(data)
    data_max = np.max(data)
    if data_max == data_min:
        return np.zeros_like(data, dtype=np.uint8)
    scaled = 255 * (data - data_min) / (data_max - data_min)
    return scaled.astype(np.uint8)

# --- Traverse patient folders ---
for patient_folder in os.listdir(root_dir):
    patient_path = os.path.join(root_dir, patient_folder)
    if not os.path.isdir(patient_path) or patient_folder == 'Images_8bits':
        continue  # Skip files or the output folder

    output_folder = os.path.join(output_root, patient_folder + '_8BITS')
    os.makedirs(output_folder, exist_ok=True)

    print(f"📁 Processing: {patient_folder}")

    for filename in os.listdir(patient_path):
        if not filename.endswith('.nii') and not filename.endswith('.nii.gz'):
            continue

        input_path = os.path.join(patient_path, filename)
        output_path = os.path.join(output_folder, filename)

        try:
            img = nib.load(input_path)
            data = img.get_fdata()
            affine = img.affine
            header = img.header

            scaled_data = scale_to_uint8(data)
            header.set_data_dtype(np.uint8)
            img_8bit = nib.Nifti1Image(scaled_data, affine, header)
            nib.save(img_8bit, output_path)

            print(f"✅ Converted: {filename}")
        except Exception as e:
            print(f"❌ Error with {filename}: {e}")

print("\n🏁 8-bit conversion complete. The images are saved in 'Images_8bits'")


In [ ]:
import nibabel as nib
import numpy as np

# --- Image path ---
image_path = 'AAA417_T2_image_uint8_2.nii.gz'  # Change this to your actual file path

# --- Load the image ---
img = nib.load(image_path)
data = img.get_fdata()
dtype = data.dtype

# --- Alternatively, check raw dtype from the original image object ---
raw_dtype = img.get_data_dtype()

# --- Verification ---
print(f"📦 Raw dtype from file: {raw_dtype}")
print(f"🧠 Data dtype after loading: {dtype}")

if raw_dtype == np.uint8:
    print("✅ The image is in 8-bit format (uint8).")
else:
    print("❌ The image is NOT in 8-bit format. It is:", raw_dtype)